In [1]:
import pandas as pd
import numpy as np
import sys, os , time, pickle
from tabulate import tabulate
from IPython.display import display

def find_match(string,list_des):
    return [value for value in list_des if string in value]

def plot_files(file_name):
    print("\nFile to show: ",file_name)
    table = pd.read_csv(file_name)
    display(table)
    #print(tabulate(table, headers='keys', tablefmt='rst')) #latex
    
def calculate_aux(archivo,what_to_do="mv",model=""):
    try:
        f = open(archivo)
        result_list = []
        for line in f.readlines():
            if what_to_do == "mv":
                if "ACC MV" in line: 
                    result_list.append(float(line.split(" ")[-1].strip()))
            elif what_to_do == 'epochs':
                if "Epochs to converge" in line and model in line: 
                    result_list.append(float(line.split(" ")[-1].strip()))
        f.close()
        return result_list
    except:
        return "-"
    
def compare_table(tipo,repre="normal", std=False):
    t = pd.DataFrame()
    t["Methods"] = ["Ann(tr)"]+methods_list
    t["Train"] = ["-"] + ["-" for _ in range(len(methods_list))]
    t["Test"] = ["-"] + ["-" for _ in range(len(methods_list))]
    if tipo == 'acc':
        try:
            t.loc[0,"Train"] = np.mean(calculate_aux(folder+"results.txt",what_to_do="mv"))
        except:
            try:
                t.loc[0,"Train"] = np.mean(calculate_aux(folder+"results_ind.txt",what_to_do="mv"))
            except:
                print("No se encontró acc")
            
    for k, method in enumerate(methods_list):
        aux = find_match(method,files)
        if tipo == "acc": #como normal?
            if std:
                test_aux = find_match("_train_std.csv",aux)
                t.loc[k+1,"Train"] = round(pd.read_csv(folder+test_aux[-1])["Accuracy"][0], 4)
                train_aux = find_match("_test_std.csv",aux)
                t.loc[k+1,"Test"] = round(pd.read_csv(folder+train_aux[-1])["Accuracy"][0], 4)
            else:
                test_aux = find_match("_train.csv",aux)
                t.loc[k+1,"Train"] = round(pd.read_csv(folder+test_aux[-1])["Accuracy"][0], 4)
                train_aux = find_match("_test.csv",aux)
                t.loc[k+1,"Test"] = round(pd.read_csv(folder+train_aux[-1])["Accuracy"][0], 4)

        elif "norm" in tipo.lower() or "js" in tipo.lower():
            if std:
                kl_aux = find_match("_train_std.csv",aux)
            else:
                kl_aux = find_match("_train.csv",aux)
            
            if len(kl_aux) != 0:                
                if repre=="R":
                    to_extract = "("+repre+") "+tipo+ " mean"
                elif repre=='G':
                    to_extract = "("+repre+") "+tipo
                try:
                    t.loc[k+1,"Train"] = round(pd.read_csv(folder+kl_aux[-1])[to_extract][0] ,4)
                except:
                    pass
        elif tipo == "epochs":
            methods_list_aux = ["soft-MV","hard-MV", "D&S", "Raykar", "Ours Global","Ours Individual","2", "3"]
            t.loc[k+1,"Train"]=calculate_aux(folder+"results.txt",what_to_do="epochs",model=methods_list_aux[k])
    return t

def get_value(lista, name, std=False):
    i = 0
    if std:
        i = 1
    return np.asarray([value[i][name][0] if type(value[i])== pd.DataFrame else -9999 for value in lista] )

def create_df(method_l,T_try, value, file_re = "./results.txt"):
    t = pd.DataFrame(np.zeros((len(method_l)+1,len(T_try))))    
    t.columns = T_try
    t.index = ["Ann(tr)"]+method_l
    t_std = t.copy()
    
    if value.lower() == "accuracy" or "F1" in value.lower():
        if len(calculate_aux(file_re,what_to_do="mv")) > len(T_try) +1:
            aux2 = calculate_aux(file_re,what_to_do="mv")
            aux = [np.mean(aux2[i:30*(i+1)]) for i in range(0,len(aux2),30)]
        else:
            aux = calculate_aux(file_re,what_to_do="mv")
        t.loc["Ann(tr)",:] = aux
        for method_l in methods_list:
            t.loc[method_l,:] = get_value(dic_res[method_l]["test"], value)
            t_std.loc[method_l,:] = get_value(dic_res[method_l]["test"], value, std=True)

    elif "inertia" in value.lower() or "iner" in value.lower():
        for method_l in methods_list:
            if "Ours" in method_l:
                t.loc[method_l,:] = get_value(dic_res[method_l]["testAux"], value)
                
    else:
        t.index = ["D_t"]+method_l
        t.loc["D_t",:] = D_t
        for method_l in methods_list:
            t.loc[method_l,:] = get_value(dic_res[method_l]["train"], value)
            t_std.loc[method_l,:] = get_value(dic_res[method_l]["train"], value, std=True)
    return t.round(4), t_std.round(4)

!ls

LabelMe  LabelMe_v2  Plot_Results.ipynb  simple_T


In [13]:
!ls LabelMe
folder = "LabelMe_v2/"

files = sorted(os.listdir(folder))
methods_list = list(set([f.split("_")[1] for f in os.listdir(folder) if (len(f.split("_")) >2) and ("UpperBound" not in f)]))
methods_list

LabelMe_DS_test.csv
LabelMe_DS_test_std.csv
LabelMe_DS_train.csv
LabelMe_DS_train_std.csv
LabelMe_hardMV_test.csv
LabelMe_hardMV_test_std.csv
LabelMe_hardMV_train.csv
LabelMe_hardMV_train_std.csv
LabelMe_OursGlobal_testAux.csv
LabelMe_OursGlobal_testAux_std.csv
LabelMe_OursGlobal_test.csv
LabelMe_OursGlobal_test_std.csv
LabelMe_OursGlobal_trainAnn.csv
LabelMe_OursGlobal_trainAnn_std.csv
LabelMe_OursGlobal_train.csv
LabelMe_OursGlobal_train_std.csv
LabelMe_OursIndividualK_testAux.csv
LabelMe_OursIndividualK_testAux_std.csv
LabelMe_OursIndividualK_test.csv
LabelMe_OursIndividualK_test_std.csv
LabelMe_OursIndividualK_trainAnn.csv
LabelMe_OursIndividualK_trainAnn_std.csv
LabelMe_OursIndividualK_train.csv
LabelMe_OursIndividualK_train_std.csv
LabelMe_OursIndividualT_testAux.csv
LabelMe_OursIndividualT_testAux_std.csv
LabelMe_OursIndividualT_test.csv
LabelMe_OursIndividualT_test_std.csv
LabelMe_OursIndividualT_trainAnn.csv
LabelMe_OursIndividualT_trainAnn_std.csv
LabelMe_OursIndividualT_trai

['DS',
 'OursIndividualK',
 'softMV',
 'OursGlobal',
 'OursIndividualT',
 'Raykar',
 'hardMV']

In [14]:
t_acc = compare_table("acc")
t_acc_std = compare_table("acc", std=True)

t_norm_R = compare_table("NormF","R")
t_norm_R_std = compare_table("NormF","R", std=True)
t_js_R = compare_table("JS","R")
t_js_R_std = compare_table("JS","R",  std=True)

t_norm_G = compare_table("NormF","G")
t_norm_G_std = compare_table("NormF","G", std=True)
t_js_G = compare_table("JS","G")
t_js_G_std = compare_table("JS","G", std=True)

t_epochs = compare_table("epochs")

In [15]:
t_acc.merge(t_acc_std.iloc[:,1:], left_index=True, right_index=True)

Methods Train_x  Test_x Train_y  Test_y
0          Ann(tr)   0.769       -   0.769       -
1               DS  0.1267  0.1412  0.0161  0.0142
2  OursIndividualK  0.7896  0.7892  0.0278  0.0318
3           softMV  0.8317  0.8173  0.0065  0.0098
4       OursGlobal  0.8432  0.8251  0.0097  0.0134
5  OursIndividualT  0.8065  0.8015  0.0273  0.0303
6           Raykar  0.7788  0.7591  0.0318  0.0314
7           hardMV  0.8041  0.7982  0.0065  0.0107

In [14]:
from IPython.display import display
display(t_js_R.merge(t_js_R_std.iloc[:,1:], left_index=True, right_index=True))
display(t_norm_R.merge(t_norm_R_std.iloc[:,1:], left_index=True, right_index=True))

Methods Train_x Test_x Train_y Test_y
0          Ann(tr)       -      -       -      -
1  OursIndividual2  0.3587      -  0.0041      -
2  OursIndividual3  0.2992      -  0.0159      -
3           softMV  0.4632      -       0      -
4           hardMV  0.4924      -       0      -

Methods Train_x Test_x Train_y Test_y
0          Ann(tr)       -      -       -      -
1  OursIndividual2   0.232      -  0.0015      -
2  OursIndividual3  0.2175      -  0.0062      -
3           softMV  0.2636      -       0      -
4           hardMV  0.2739      -       0      -

In [26]:
display(t_js_G.merge(t_js_G_std.iloc[:,1:], left_index=True, right_index=True))
display(t_norm_G.merge(t_norm_G_std.iloc[:,1:], left_index=True, right_index=True))

Methods Train_x Test_x Train_y Test_y
0          Ann(tr)       -      -       -      -
1           softMV  0.0242      -       0      -
2  OursIndividual2  0.0512      -  0.0032      -
3           hardMV  0.0347      -       0      -
4  OursIndividual3  0.0517      -   0.004      -

Methods Train_x Test_x Train_y Test_y
0          Ann(tr)       -      -       -      -
1           softMV  0.0566      -       0      -
2  OursIndividual2  0.0545      -  0.0026      -
3           hardMV  0.0736      -       0      -
4  OursIndividual3  0.0674      -  0.0036      -

## T change

In [2]:
folder = "./simple_T/"

In [3]:
to_check = [100,500,1500,3500,6000,10000]

if 'simple' in folder.lower():
    T_data = 5 #for now is this..
    N = 2500
elif "cifar" in folder.lower():
    T_data = 3
    N = 50000
D_t = np.asarray([N*T_data/T for T in to_check])

files_f = [f for f in os.listdir(folder) if ".pickle" in f]
dic_res = {}
for file in files_f:
    #if len(file.split("_")) >2:
    _, method_name, method_type = (file.split(".pickle")[0]).split("_")
    with open(folder+file, 'rb') as handle:
        if method_name in dic_res:
            dic_res[method_name][method_type] = pickle.load(handle)
        else:
            dic_res[method_name] = {}
            dic_res[method_name][method_type] = pickle.load(handle)
methods_list = list(dic_res.keys())
methods_list

['softMV',
 'OursGlobal',
 'DS',
 'OursIndividualT',
 'hardMV',
 'Raykar',
 'OursIndividualK']

In [5]:
t, t_std = create_df(methods_list, to_check,"Accuracy", file_re=folder+'results.txt')
t.merge(t_std.iloc[:,1:], left_index=True, right_index=True)

100   500_x  1500_x  3500_x  6000_x  10000_x   500_y  \
Ann(tr)          0.7310  0.7131  0.7436  0.7375  0.7114   0.7127  0.0000   
softMV           0.6984  0.6752  0.6948  0.6885  0.6558   0.6786  0.0293   
OursGlobal       0.8918  0.9092  0.8910  0.9160  0.8790   0.8998  0.0160   
DS               0.9451  0.9364  0.9211  0.9230  0.8969   0.8744  0.0099   
OursIndividualT  0.9461  0.9427  0.9059  0.9457  0.9433   0.8643  0.0045   
hardMV           0.8373  0.7998  0.8163  0.8296  0.7783   0.8062  0.0255   
Raykar           0.9397  0.9375  0.9305  0.9053  0.7142   0.7375  0.0049   
OursIndividualK  0.9437  0.9454  0.9465  0.9190  0.9186   0.7763  0.0035   

                 1500_y  3500_y  6000_y  10000_y  
Ann(tr)          0.0000  0.0000  0.0000   0.0000  
softMV           0.0344  0.0350  0.0178   0.0305  
OursGlobal       0.1146  0.0108  0.1182   0.0105  
DS               0.0139  0.0188  0.0264   0.0499  
OursIndividualT  0.0217  0.0040  0.0045   0.0494  
hardMV           0.0470  0.0432  0.0583   0.0478  
Raykar           0.0098  0.0381  0.1351   0.1340  
OursIndividualK  0.0049  0.0252  0.0205   0.0839

In [6]:
t, t_std = create_df(methods_list,to_check, "(R) JS mean")
#t, t_std = create_df(methods_list,to_check, "(R) NormF mean")
t.merge(t_std.iloc[:,1:], left_index=True, right_index=True)

100   500_x  1500_x  3500_x  6000_x  10000_x   500_y  \
softMV           0.0861  0.0863  0.0864  0.0877  0.0870   0.0872  0.0000   
OursGlobal       0.1113  0.1105  0.1157  0.1237  0.1253   0.1159  0.0141   
DS               0.0107  0.0655  0.1805  0.2528  0.2800   0.3015  0.0000   
OursIndividualT  0.0177  0.0205  0.0548  0.1125  0.1507   0.1870  0.0014   
hardMV           0.0851  0.0874  0.0847  0.0861  0.0860   0.0868  0.0000   
Raykar           0.0142  0.0559  0.1138  0.1548  0.1744   0.1811  0.0004   
OursIndividualK  0.0172  0.0186  0.0476  0.0930  0.1330   0.1835  0.0026   

                 1500_y  3500_y  6000_y  10000_y  
softMV           0.0000  0.0000  0.0000   0.0000  
OursGlobal       0.0351  0.0142  0.0265   0.0096  
DS               0.0000  0.0000  0.0000   0.0000  
OursIndividualT  0.0053  0.0028  0.0031   0.0137  
hardMV           0.0000  0.0000  0.0000   0.0000  
Raykar           0.0005  0.0010  0.0041   0.0036  
OursIndividualK  0.0056  0.0052  0.0071   0.0152

In [7]:
t, t_std = create_df(methods_list,to_check,"(G) JS")
#t, t_std = create_df(methods_list,to_check,"(G) NormF")
t.merge(t_std.iloc[:,1:], left_index=True, right_index=True)

100   500_x  1500_x  3500_x  6000_x  10000_x   500_y  \
softMV           0.0139  0.0140  0.0141  0.0137  0.0151   0.0141  0.0000   
OursGlobal       0.0510  0.0475  0.0611  0.0618  0.0653   0.0571  0.0138   
DS               0.0001  0.0025  0.0053  0.0227  0.0401   0.0483  0.0000   
OursIndividualT  0.0036  0.0017  0.0103  0.0046  0.0089   0.0263  0.0006   
hardMV           0.0155  0.0182  0.0143  0.0140  0.0167   0.0176  0.0000   
Raykar           0.0021  0.0218  0.0587  0.0913  0.1076   0.1090  0.0003   
OursIndividualK  0.0061  0.0025  0.0024  0.0086  0.0132   0.0301  0.0014   

                 1500_y  3500_y  6000_y  10000_y  
softMV           0.0000  0.0000  0.0000   0.0000  
OursGlobal       0.0349  0.0146  0.0246   0.0102  
DS               0.0000  0.0000  0.0000   0.0000  
OursIndividualT  0.0043  0.0006  0.0004   0.0096  
hardMV           0.0000  0.0000  0.0000   0.0000  
Raykar           0.0003  0.0007  0.0024   0.0021  
OursIndividualK  0.0016  0.0023  0.0012   0.0100

In [ ]:
print(tabulate(t,headers='keys',tablefmt='latex'))

In [8]:
t, _ = create_df(methods_list,to_check,"Iner JS")
#t, _ = create_df(methods_list,to_check,"Iner NormF")
t.merge(t_std.iloc[:,1:], left_index=True, right_index=True)

100   500_x  1500_x  3500_x  6000_x  10000_x   500_y  \
Ann(tr)          0.0000  0.0000  0.0000  0.0000  0.0000   0.0000  0.0000   
softMV           0.0000  0.0000  0.0000  0.0000  0.0000   0.0000  0.0000   
OursGlobal       0.2539  0.2087  0.1349  0.2122  0.2082   0.2062  0.0138   
DS               0.0000  0.0000  0.0000  0.0000  0.0000   0.0000  0.0000   
OursIndividualT  0.3096  0.3609  0.3964  0.6011  0.6713   0.7111  0.0006   
hardMV           0.0000  0.0000  0.0000  0.0000  0.0000   0.0000  0.0000   
Raykar           0.0000  0.0000  0.0000  0.0000  0.0000   0.0000  0.0003   
OursIndividualK  0.2976  0.3421  0.3874  0.5060  0.5801   0.6641  0.0014   

                 1500_y  3500_y  6000_y  10000_y  
Ann(tr)          0.0000  0.0000  0.0000   0.0000  
softMV           0.0000  0.0000  0.0000   0.0000  
OursGlobal       0.0349  0.0146  0.0246   0.0102  
DS               0.0000  0.0000  0.0000   0.0000  
OursIndividualT  0.0043  0.0006  0.0004   0.0096  
hardMV           0.0000  0.0000  0.0000   0.0000  
Raykar           0.0003  0.0007  0.0024   0.0021  
OursIndividualK  0.0016  0.0023  0.0012   0.0100